## Шаблонный метод (Template method)

**Шаблонный метод** - это поведенческий паттерн проектирования, который определяет скелет алгоритма, перекладывая ответственность за некоторые его шаги на подклассы. Паттерн позволяет подклассам переопределять шаги алгоритма, не меняя его общей структуры. 
#### Применимость:
* Когда подклассы должны расширять базовый алгоритм, не меняя его структуры.
* Когда у вас есть несколько классов, делающих одно и то же с незначительными отличиями. Если вы редактируете один класс, то приходится вносить такие же правки и в остальные классы.  

#### Достоинства метода:
* Локализация и вычленение общего для нескольких классов кода для избегания дублирования.  

#### Недостатки метода:
* Жесткая ограниченность существующим скелетом.
* Сложно поддерживать с ростом количества шагов.

Ниже представлена реализация метода на примере расчета скважины (расчет угла наклона ствола и забойного давления). Угол рассчитывается для всех типов скважины одинаково, расчет же забойного давления определяется конкретным способом эксплуатации.

In [1]:
from abc import ABC, abstractmethod
import math

class Well(ABC):
    """
    Асбтрактный класс - некая скважина
    """

    def __init__(self, md, tvd, wh_pressure, q_liq, gas_inj=None):
        self.md = md
        self.tvd = tvd
        self.wh_pressure = wh_pressure
        self.q_liq = q_liq
        self.gas_inj = gas_inj
        self.angle = None

    def calc_well(self):
        """
        Шаблонный метод.
        Скелет расчетного алгоритма
        """
        print("Запущен расчет скважины")
        self._calc_inclinometry()
        self.calc_bh_pressure()

    def _calc_inclinometry(self):
        """
        Расчет вертикального угла скважины.
        Обязательный шаг с реализацией по умолчанию
        """
        if self.md == self.tvd:
            angle = 0
        else:
            angle = math.degrees(math.acos(self.tvd / self.md))
        print(f"Угол наклона скважины равен {angle}")

    @abstractmethod
    def calc_bh_pressure(self):
        """
        Расчет забойного давления.
        Обязательный шаг с переопределением метода в зависимости от конкретного подкласса
        """
        pass


class Esp(Well):
    """Конкретный класс - скважина ЭЦН"""

    def calc_bh_pressure(self):
        print("Скважина оборудована ЭЦН")
        dp_esp = self.q_liq * 0.1
        bh_pressure = self.wh_pressure + 9.81 * 0.85 * self.tvd / 100 - dp_esp
        print(f"Перепад давления в насосе {dp_esp} атм")
        print(f"Забойное давление {bh_pressure} атм")


class Gaslift(Well):
    """Конкретный класс - газлифтная скважина"""

    def calc_bh_pressure(self):
        if self.gas_inj is None or self.gas_inj == 0:
            print("Скважина оборудована фонтаном")
            bh_pressure = self.wh_pressure + 9.81 * 0.85 * self.tvd / 100
        else:
            print("Скважина оборудована газлифтом")
            bh_pressure = self.wh_pressure + 9.81 * (0.85 / self.gas_inj) * self.tvd / 100
        print(f"Забойное давление {bh_pressure} атм")


In [2]:
#Исходные данные
md = 3000
tvd = 2500
wh_pressure = 10
q_liq = 200
qgas_inj = 2

# Расчет ЭЦН
calc = Esp(md, tvd, wh_pressure, q_liq)
result = calc.calc_well()

#Расчет газлифта
# calc = Gaslift(md, tvd, wh_pressure, q_liq, qgas_inj)
# result = calc.calc_well()

Запущен расчет скважины
Угол наклона скважины равен 33.55730976192071
Скважина оборудована ЭЦН
Перепад давления в насосе 20.0 атм
Забойное давление 198.4625 атм
